In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import split
from pyspark.sql.functions import isnan, when, count
from pyspark.sql.functions import initcap, col
import re
import string

In [0]:
dbutils.fs.ls('/FileStore/tables')

Out[104]: [FileInfo(path='dbfs:/FileStore/tables/housing_1.csv', name='housing_1.csv', size=1335847, modificationTime=1660127283000),
 FileInfo(path='dbfs:/FileStore/tables/housing_2.csv', name='housing_2.csv', size=1281598, modificationTime=1660127601000),
 FileInfo(path='dbfs:/FileStore/tables/housing_3.csv', name='housing_3.csv', size=1518191, modificationTime=1660127675000),
 FileInfo(path='dbfs:/FileStore/tables/synthetic_titanic.csv', name='synthetic_titanic.csv', size=5949705, modificationTime=1659529715000)]

In [0]:
## TASK 1 ## 
## COMPLETE THE FOLLOWING DATA DICTIONARY
'''
No. - the index of the table
MLSID - Agent ID
StreetNumberNumeric - the house number
streetaddress- Address
STREETNAME - ''
PostalCode - Postal code
StateOrProvince - ''
City- ''
SalesClosePrice -  Closing price 
LISTDATE - date it was listed
LISTPRICE - price 
LISTTYPE - type of list 
OriginalListPrice - price when first listed
PricePerTSFT - Unknown 
Foundation size - Size of foundation
Fence - Fence type
MapLetter - Location on a map and the surround geographic properties 
LotSizeDimensions -  Size dimensions of the lot 
SchoolDistrictNumber - ''
DAYSONMARKET - ''
offmarketdate - Date left market 
Fireplaces - ''
RoomAreaN - Size of room 
roomtype - ''
ROOF - ''
RoomFloorN - Location of the room 
PotentialShortSale - If the house will have a short sale 
PoolDescription - ''
PDOM - Property days on the market 
GarageDescription -'' 
SQFTABOVEGROUND - Squarefoot above ground
Taxes - ''
TAXWITHASSESSMENTS - ''
TAXYEAR - ''
LivingArea - ''
UNITNUMBER - ''
YEARBUILT- ''     
ZONING - House zoning/residential type
STYLE - style of house
ACRES - ''
CoolingDescription- Heating type 
APPLIANCES - ''
backonmarketdate - ''
ROOMFAMILYCHAR - 
EXTERIOR - Material of exterior 
DiningRoomDescription - ''
BASEMENT - Basement type
BathsFull/BathsHalf/BATHQUARTER/BATHSTHREEQUARTER- Bathroom types
Class - Class type of house 
BATHSTOTAL - Total number of bathroom 
BATHDESC - Bathroom description
Bedrooms - Number of bedrooms
SQFTBELOWGROUND - Square foot below ground
AssumableMortgage - is the mortgage assumable
AssociationFee - Fee for promotion
ASSESSMENTPENDING - Is the assessment pending
AssessedValuation -'''


Out[105]: "\nNo. - the index of the table\nMLSID - Agent ID\nStreetNumberNumeric - the house number\nstreetaddress- Address\nSTREETNAME - ''\nPostalCode - Postal code\nStateOrProvince - ''\nCity- ''\nSalesClosePrice -  Closing price \nLISTDATE - date it was listed\nLISTPRICE - price \nLISTTYPE - type of list \nOriginalListPrice - price when first listed\nPricePerTSFT - Unknown \nFoundation size - Size of foundation\nFence - Fence type\nMapLetter - Location on a map and the surround geographic properties \nLotSizeDimensions -  Size dimensions of the lot \nSchoolDistrictNumber - ''\nDAYSONMARKET - ''\noffmarketdate - Date left market \nFireplaces - ''\nRoomAreaN - Size of room \nroomtype - ''\nROOF - ''\nRoomFloorN - Location of the room \nPotentialShortSale - If the house will have a short sale \nPoolDescription - ''\nPDOM - Property days on the market \nGarageDescription -'' \nSQFTABOVEGROUND - Squarefoot above ground\nTaxes - ''\nTAXWITHASSESSMENTS - ''\nTAXYEAR - ''\nLivingArea - ''\

In [0]:
## DATA CLEANING##

# based on the business rules outlined in the brief, start to assemble a list of columns to drop
# once you have a final list, feel free to give them a drop
# they can be useless due to nulls, or useless because they have no predictive power, either way get rid of them!!

In [0]:
## Did not include PricePerTSFT in our schema as this column is not in the dataset 
housing_1_schema = StructType([
    StructField("No", FloatType(), True),
    StructField("MLSID", StringType(), True),
    StructField("StreetNumberNumeric", FloatType(), True),
    StructField("streetaddress", StringType(), True),
    StructField("STREETNAME", StringType(), True),
    StructField("Postalcode", FloatType(), True),
    StructField("StateOrProvince", StringType(), True),
    StructField("City", StringType(), True),
    StructField("SalesClosePrice", FloatType(), True),
    StructField("LISTDATE", StringType(), True),
    StructField("LISTPRICE", FloatType(), True),
    StructField("LISTTYPE", StringType(), True),
    StructField("OriginalListPrice", FloatType(), True),
    StructField("Foundation Size", FloatType(), True),
    StructField("Fence", StringType(), True),
    StructField("MapLetter", StringType(), True),
    StructField("LotSizeDimensions", StringType(), True),
    StructField("SchoolDistrictNumber", StringType(), True),
    StructField("DAYSONMARKET", FloatType(), True),
    StructField("offmarketdate", StringType(), True),
    StructField("Fireplaces", FloatType(), True),
    StructField("RoomArea4", StringType(), True),
    StructField("roomtype", StringType(), True),
    StructField("ROOF", StringType(), True),
    StructField("RoomFloor4", StringType(), True),
    StructField("PotentialShortSale", StringType(), True),
    StructField("PoolDescription", StringType(), True),
    StructField("PDOM", FloatType(), True),
    StructField("GarageDescription", StringType(), True),
    StructField("SQFTABOVEGROUND", FloatType(), True),
    StructField("Taxes", FloatType(), True),
    StructField("RoomFloor1", StringType(), True),
    StructField("RoomArea1", StringType(), True),
    StructField("TAXWITHASSESSMENTS", FloatType(), True),
    StructField("TAXYEAR", StringType(), True),
    StructField("LivingArea", FloatType(), True),
    StructField("UNITNUMBER", StringType(), True),
    StructField("YEARBUILT", StringType(), True),
    StructField("ZONING", StringType(), True),
    StructField("STYLE", StringType(), True),
    StructField("ACRES", FloatType(), True),
    StructField("CoolingDescription", StringType(), True),
    StructField("APPLIANCES", StringType(), True),
    StructField("backonmarketdate", StringType(), True),
    StructField("ROOMFAMILYCHAR", StringType(), True),
    StructField("RoomArea3", StringType(), True),
    StructField("EXTERIOR", StringType(), True),
    StructField("RoomFloor3", StringType(), True),
    StructField("RoomFloor2", StringType(), True),
    StructField("RoomArea2", StringType(), True),
    StructField("DiningRoomDescription", StringType(), True),
    StructField("BASEMENT", StringType(), True),
    StructField("BathsFull", FloatType(), True),
    StructField("BathsHalf", FloatType(), True),
    StructField("BATHQUARTER", FloatType(), True),
    StructField("BATHSTHREEQUARTER", FloatType(), True),
    StructField("Class", StringType(), True),
    StructField("BATHSTOTAL", FloatType(), True),
    StructField("BATHDESC", StringType(), True),
    StructField("RoomArea5", StringType(), True),
    StructField("RoomFloor5", StringType(), True),
    StructField("RoomArea6", StringType(), True),
    StructField("RoomFloor6", StringType(), True),
    StructField("RoomArea7", StringType(), True),
    StructField("RoomFloor7", StringType(), True),
    StructField("RoomArea8", StringType(), True),
    StructField("RoomFloor8", StringType(), True),
    StructField("Bedrooms", FloatType(), True),
    StructField("SQFTBELOWGROUND", FloatType(), True),
    StructField("AssumableMortgage", StringType(), True),
    StructField("AssociationFee", FloatType(), True),
    StructField("ASSESSMENTPENDING", StringType(), True),
    StructField("AssessedValuation", FloatType(), True),
])
housing_1=spark.read.csv('/FileStore/tables/housing_1.csv', header=True, schema=housing_1_schema)
housing_2=spark.read.csv('/FileStore/tables/housing_2.csv', header=True, schema=housing_1_schema)
housing_3=spark.read.csv('/FileStore/tables/housing_3.csv', header=True, schema=housing_1_schema)

In [0]:
## Combining the dataframes 
housing = housing_1.unionByName(housing_2)
housing_1 = housing.unionByName(housing_3)

In [0]:
## Convert to a pandas dataframe to check for nulls as it will be easier to read 
housing_null = housing_1.toPandas()
housing_null.isnull().sum()


Out[109]: No                        0
MLSID                     0
StreetNumberNumeric       0
streetaddress             0
STREETNAME                0
                       ... 
SQFTBELOWGROUND           0
AssumableMortgage      3104
AssociationFee            0
ASSESSMENTPENDING         0
AssessedValuation         0
Length: 73, dtype: int64

In [0]:
## This shows how many none null values a cloumn contains 
housing_null.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 73 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   No                     5000 non-null   float32
 1   MLSID                  5000 non-null   object 
 2   StreetNumberNumeric    5000 non-null   float32
 3   streetaddress          5000 non-null   object 
 4   STREETNAME             5000 non-null   object 
 5   Postalcode             5000 non-null   float32
 6   StateOrProvince        5000 non-null   object 
 7   City                   5000 non-null   object 
 8   SalesClosePrice        5000 non-null   float32
 9   LISTDATE               5000 non-null   object 
 10  LISTPRICE              5000 non-null   float32
 11  LISTTYPE               5000 non-null   object 
 12  OriginalListPrice      5000 non-null   float32
 13  Foundation Size        5000 non-null   float32
 14  Fence                  2972 non-null   object 
 15  MapL

In [0]:
## Function to drop columns with 50% or more missing values 
def dropcols(data):
    for col in data.columns:
        data.dropna(thresh=data.shape[0]*0.5,how='all',axis=1, inplace=True)
        
    return data
dropcols(housing_null)
housing_null.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 67 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   No                     5000 non-null   float32
 1   MLSID                  5000 non-null   object 
 2   StreetNumberNumeric    5000 non-null   float32
 3   streetaddress          5000 non-null   object 
 4   STREETNAME             5000 non-null   object 
 5   Postalcode             5000 non-null   float32
 6   StateOrProvince        5000 non-null   object 
 7   City                   5000 non-null   object 
 8   SalesClosePrice        5000 non-null   float32
 9   LISTDATE               5000 non-null   object 
 10  LISTPRICE              5000 non-null   float32
 11  LISTTYPE               5000 non-null   object 
 12  OriginalListPrice      5000 non-null   float32
 13  Foundation Size        5000 non-null   float32
 14  Fence                  2972 non-null   object 
 15  MapL

In [0]:
## Return as pyspark dataframe
housing_1=spark.createDataFrame(housing_null)

In [0]:
## Dropping columns we believe are noise as there is only 1 unique value or the columns are not helpful
## These include columns that are descriptions or there are too many nulls and they cannot be filled in 
## And dropping room floor as the location of a room is not likely to massively affect the price
## Finally dropping columns that would not be helpful in a model creation, Tax column is dropped as it is similar to the tax with assessments  
housing_1 = housing_1.drop('MLSID','Fence', 'ROOMFAMILYCHAR', 'DiningRoomDescription', 'BATHDESC', 'RoomFloor1','RoomFloor2', 'RoomFloor3','RoomFloor4','RoomFloor5','RoomFloor6','RoomFloor7', 'ASSESSMENTPENDING', 'RoomArea3', 'RoomArea2', 'RoomArea7', 'AssessedValuation', 'Class', 'Taxes', 'Streetaddress', 'STREETNAME'  )

In [0]:
## Drop rows from columns with only a few missing values or rows that belong to columns that will not be possible to fill in the null but are still an important column
housing_1=housing_1.dropna(how='any',subset='APPLIANCES')
housing_1=housing_1.dropna(how='any',subset='RoomArea1')
housing_1=housing_1.dropna(how='any',subset='RoomArea4')
housing_1=housing_1.dropna(how='any',subset='RoomArea5')
housing_1=housing_1.dropna(how='any',subset='RoomArea6')

In [0]:
## Counting the number of unique types of roof
housing_1.groupBy(col('ROOF')).count().show()

+--------------------+-----+
|                ROOF|count|
+--------------------+-----+
|Asphalt Shingles,...|  599|
|Flat, Tar/Gravel,...|    1|
|Asphalt Shingles,...|  743|
|               Metal|    9|
|                null|  442|
|                Flat|   35|
|Asphalt Shingles,...|    1|
|Asphalt Shingles,...|  262|
|Asphalt Shingles,...|    1|
| Age 8 Years or Less|   85|
|Asphalt Shingles,...|    2|
|Asphalt Shingles,...|    1|
|    Asphalt Shingles| 1240|
|Asphalt Shingles,...|  272|
|Asphalt Shingles,...|    2|
|Pitched, Age Over...|    4|
|    Age Over 8 Years|   59|
| Unspecified Shingle|    4|
|Asphalt Shingles,...|  248|
|             Pitched|    5|
+--------------------+-----+
only showing top 20 rows



In [0]:
## Fill in missing valiues with most common value as nearly all of the roofs are this type
housing_1 = housing_1.fillna('Asphalt Shingles', subset='ROOF')

In [0]:
# Making a pandas dataframe to make some changes as they are easier to make, found problems doing this in a pyspark df 
housing_1p=housing_1.toPandas()


In [0]:
## Improving column names, making them all lowercase 
housing_1p.columns= housing_1p.columns.str.strip().str.lower()

In [0]:
housing_1p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   no                    4115 non-null   float32
 1   streetnumbernumeric   4115 non-null   float32
 2   postalcode            4115 non-null   float32
 3   stateorprovince       4115 non-null   object 
 4   city                  4115 non-null   object 
 5   salescloseprice       4115 non-null   float32
 6   listdate              4115 non-null   object 
 7   listprice             4115 non-null   float32
 8   listtype              4115 non-null   object 
 9   originallistprice     4115 non-null   float32
 10  foundation size       4115 non-null   float32
 11  mapletter             4115 non-null   object 
 12  lotsizedimensions     4115 non-null   object 
 13  schooldistrictnumber  4115 non-null   object 
 14  daysonmarket          4115 non-null   float32
 15  offmarketdate        

In [0]:
## Splitting the Room areas columns on the x 
#splitting the surface and volume of RoomArea4

#housing_1p = housing_1.toPandas()


# Remove special characters from the string

#housing_1p['RoomArea4'] = housing_1p['RoomArea4'].str.replace('\W', '')

# Checks the format of the data 
#for x in housing_2p['RoomArea4']:
  #  if x != r'\w\x\w':
      #  x = '0x0'





#splitted = housing_1p["RoomArea4"].str.split("x")
#splitted.head()
#creating a dataframe for surface and volume of RoomArea4
#surface4 = []
#volume4=[]
#count=0
#for i in splitted:
    #errors occurred that were too tricky to resolve within the deadline, so we use try-except to eliminate errors which resulted in 6 datapoints not being incorporated
 #   try:
  #      surface4.append(float(i[0]))
   #     volume4.append(float(i[1]))
    #    count+=1
    #except:
     #   ValueError or IndexError
      #  surface4.append(0.0) #we will need to drop these rows later
       # volume4.append(0.0)
#for x in surface4:
 #   if x == None:
  #      x = '0x0'

#for x in volume4:
 #   if x == None:
  #      x = '0x0'
#creating a dictionary with surface and volume
#sv = { "surface" : [x for x in surface4], "volume" : [x for x in volume4]}
#using the dictionary to build a dataframe
#sv_df = pd.DataFrame(data=sv)
#multiplying the columns to get the total room area
#sv_df['total']= sv_df['surface']*sv_df['volume']
#sv_df.head()
#housing_1 = spark.createDataFrame(data=housing_1_pd)
#housing_1.toPandas().dtypes 

In [0]:
## Descriptive Stats
housing_1p.describe()

,no,streetnumbernumeric,postalcode,salescloseprice,listprice,originallistprice,foundation size,daysonmarket,fireplaces,pdom,...,livingarea,acres,bathsfull,bathshalf,bathquarter,bathsthreequarter,bathstotal,bedrooms,sqftbelowground,associationfee
count,4115.000000,4115.000000,4115.000000,4.115000e+03,4.115000e+03,4.115000e+03,4115.000000,4115.000000,4115.000000,4115.000000,...,4115.000000,4115.000000,4115.000000,4115.000000,4115.000000,4115.000000,4115.000000,4115.000000,4115.000000,4115.000000
mean,2518.565674,2564.208496,55114.179688,2.608833e+05,2.612413e+05,2.741916e+05,1014.141907,28.070959,0.597084,20.765978,...,1848.528076,0.202107,1.297205,0.375942,0.026731,0.504496,2.204374,3.142892,373.976685,73.285538
std,1415.951660,2757.983887,14.158646,1.316042e+05,1.353411e+05,4.430136e+05,331.185303,27.785112,0.717848,18.899445,...,791.746460,0.348864,0.563325,0.507460,0.162817,0.605570,0.918285,0.900221,413.107513,137.246002
min,1.000000,3.000000,55042.000000,5.080000e+04,4.000000e+04,5.000000e+00,127.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,1310.500000,895.500000,55106.000000,1.770000e+05,1.750000e+05,1.750000e+05,788.000000,10.000000,0.000000,8.000000,...,1320.000000,0.100500,1.000000,0.000000,0.000000,0.000000,2.000000,2.500000,0.000000,0.000000
50%,2570.000000,1600.000000,55117.000000,2.270000e+05,2.250000e+05,2.290000e+05,968.000000,19.000000,0.000000,16.000000,...,1653.000000,0.140000,1.000000,0.000000,0.000000,0.000000,2.000000,3.000000,300.000000,0.000000
75%,3727.500000,2546.500000,55125.000000,3.100000e+05,3.090000e+05,3.150000e+05,1172.000000,38.000000,1.000000,27.000000,...,2138.000000,0.240000,2.000000,1.000000,0.000000,1.000000,3.000000,4.000000,643.500000,165.000000
max,4999.000000,11900.000000,55130.000000,1.600000e+06,1.850000e+06,2.199000e+07,3792.000000,225.000000,8.000000,195.000000,...,8222.000000,9.470000,5.000000,4.000000,2.000000,3.000000,7.000000,8.000000,2487.000000,1683.000000


In [0]:
def fixdates(data):
    data[['listdatenew','useless']] = data.listdate.str.split(" ",expand=True,)
    data[['offmarketdatenew','useless2']] = data.offmarketdate.str.split(" ",expand=True,)
    data.drop('listdate', axis=1, inplace=True)
    data.drop('offmarketdate', axis=1, inplace=True)
    data.drop('useless', axis=1, inplace=True)
    data.drop('useless2', axis=1, inplace=True)
    return data
fixdates(housing_1p) 
    

,no,streetnumbernumeric,postalcode,stateorprovince,city,salescloseprice,listprice,listtype,originallistprice,foundation size,...,bathquarter,bathsthreequarter,bathstotal,roomarea5,roomarea6,bedrooms,sqftbelowground,associationfee,listdatenew,offmarketdatenew
0,1.0,11511.0,55042.0,MN,LELM - Lake Elmo,143000.0,139900.0,Exclusive Right,139900.0,980.0,...,0.0,0.0,2.0,13 x 11,10 x 10,3.0,0.0,0.0,7/15/2017,7/30/2017
1,2.0,11200.0,55042.0,MN,LELM - Lake Elmo,190000.0,210000.0,Exclusive Right,210000.0,1144.0,...,0.0,2.0,3.0,15x11,14x11,4.0,960.0,0.0,10/9/2017,10/13/2017
2,3.0,8583.0,55042.0,MN,LELM - Lake Elmo,225000.0,225000.0,Exclusive Right,225000.0,1102.0,...,0.0,0.0,1.0,12x12,12x12,2.0,0.0,0.0,6/26/2017,7/24/2017
3,5.0,2915.0,55042.0,MN,LELM - Lake Elmo,249900.0,239900.0,Exclusive Right,239900.0,1026.0,...,0.0,1.0,2.0,13x11,13x12,3.0,1026.0,0.0,9/12/2017,10/3/2017
4,6.0,3604.0,55042.0,MN,LELM - Lake Elmo,255000.0,239900.0,Exclusive Right,239900.0,1152.0,...,0.0,0.0,2.0,25.5 x 11,10 x 11,3.0,0.0,0.0,4/10/2017,4/27/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4110,4994.0,10869.0,55129.0,MN,WB - Woodbury,795000.0,799000.0,Exclusive Right,799000.0,1911.0,...,0.0,1.0,4.0,18x16,17x12,6.0,2295.0,80.0,7/21/2017,9/18/2017
4111,4995.0,10271.0,55129.0,MN,WB - Woodbury,780000.0,799000.0,Exclusive Right,839000.0,1882.0,...,0.0,0.0,4.0,20x15,18x15,5.0,1896.0,480.0,6/21/2017,8/14/2017
4112,4997.0,10746.0,55129.0,MN,WB - Woodbury,805000.0,850000.0,Exclusive Right,875000.0,2290.0,...,0.0,1.0,5.0,32x36,11x12,5.0,2220.0,80.0,8/1/2017,12/12/2017
4113,4998.0,10721.0,55129.0,MN,WB - Woodbury,865000.0,899999.0,Exclusive Right,949900.0,2096.0,...,0.0,2.0,5.0,19x15,13x12,4.0,1861.0,86.0,3/27/2017,8/22/2017


In [0]:
housing_1p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   no                    4115 non-null   float32
 1   streetnumbernumeric   4115 non-null   float32
 2   postalcode            4115 non-null   float32
 3   stateorprovince       4115 non-null   object 
 4   city                  4115 non-null   object 
 5   salescloseprice       4115 non-null   float32
 6   listprice             4115 non-null   float32
 7   listtype              4115 non-null   object 
 8   originallistprice     4115 non-null   float32
 9   foundation size       4115 non-null   float32
 10  mapletter             4115 non-null   object 
 11  lotsizedimensions     4115 non-null   object 
 12  schooldistrictnumber  4115 non-null   object 
 13  daysonmarket          4115 non-null   float32
 14  fireplaces            4115 non-null   float32
 15  roomarea4            

In [0]:
## Converting back to pyspark 
housing_1=spark.createDataFrame(housing_1p)

In [0]:
## Checking data types 
housing_1.printSchema()

root
 |-- no: float (nullable = true)
 |-- streetnumbernumeric: float (nullable = true)
 |-- postalcode: float (nullable = true)
 |-- stateorprovince: string (nullable = true)
 |-- city: string (nullable = true)
 |-- salescloseprice: float (nullable = true)
 |-- listprice: float (nullable = true)
 |-- listtype: string (nullable = true)
 |-- originallistprice: float (nullable = true)
 |-- foundation size: float (nullable = true)
 |-- mapletter: string (nullable = true)
 |-- lotsizedimensions: string (nullable = true)
 |-- schooldistrictnumber: string (nullable = true)
 |-- daysonmarket: float (nullable = true)
 |-- fireplaces: float (nullable = true)
 |-- roomarea4: string (nullable = true)
 |-- roomtype: string (nullable = true)
 |-- roof: string (nullable = true)
 |-- potentialshortsale: string (nullable = true)
 |-- pdom: float (nullable = true)
 |-- garagedescription: string (nullable = true)
 |-- sqftaboveground: float (nullable = true)
 |-- roomarea1: string (nullable = true)
 |--

In [0]:
## back to rooms for a sec
## can we work out what each room is?
## Let's start by getting some of the options
## select the first 5 rows from room type

In [0]:
# Selecting first 5 rows from room type 
housing_1.select(col('roomtype')).show(5)

+--------------------+
|            roomtype|
+--------------------+
|Living Room, Dini...|
|Living Room, Dini...|
|Living Room, Dini...|
|Living Room, Dini...|
|Living Room, Dini...|
+--------------------+
only showing top 5 rows



In [0]:
## since it looks like they're in the same order, maybe its like room 1 = living room?
## check that below
## if there is a mapping, rename the room columns accordingly

In [0]:
## Rooms 1-5 do seem to follow a pattern of what room type they could be but it is not always the case. It would not be possible
## to determine which room type rooms 6-8 are 
## However we dropped room 3 

In [0]:
## Only renamed some of the rooms as we dropped some and any rooms after room 5 cannot be distinguished as it changes every time 
housing_1 = housing_1.withColumnRenamed('RoomArea1', 'living room area')
housing_1 = housing_1.withColumnRenamed('RoomArea2', 'dining room area')
housing_1 = housing_1.withColumnRenamed('RoomArea4', 'kitchen area')
housing_1 = housing_1.withColumnRenamed('RoomArea5', 'bedroom 1 area')

In [0]:
housing_1.columns

Out[131]: ['no',
 'streetnumbernumeric',
 'postalcode',
 'stateorprovince',
 'city',
 'salescloseprice',
 'listprice',
 'listtype',
 'originallistprice',
 'foundation size',
 'mapletter',
 'lotsizedimensions',
 'schooldistrictnumber',
 'daysonmarket',
 'fireplaces',
 'kitchen area',
 'roomtype',
 'roof',
 'potentialshortsale',
 'pdom',
 'garagedescription',
 'sqftaboveground',
 'living room area',
 'taxwithassessments',
 'taxyear',
 'livingarea',
 'yearbuilt',
 'zoning',
 'style',
 'acres',
 'coolingdescription',
 'appliances',
 'exterior',
 'basement',
 'bathsfull',
 'bathshalf',
 'bathquarter',
 'bathsthreequarter',
 'bathstotal',
 'bedroom 1 area',
 'roomarea6',
 'bedrooms',
 'sqftbelowground',
 'associationfee',
 'listdatenew',
 'offmarketdatenew']